In [1]:
import pyaudio
import numpy as np
import librosa
import threading
import tkinter as tk
from pythonosc import udp_client

# Initially set a default tempo, e.g., 120 BPM
beat_duration = 60 / 120
current_note = None  # Variable to hold the current note

# Flag to control audio processing thread
shutdown_event = threading.Event()

# PyAudio configuration
FORMAT = pyaudio.paFloat32
CHANNELS = 1
RATE = 44100
CHUNK = 2048

# OSC Configuration
OSC_IP = "127.0.0.1"
OSC_PORT = 6448
OSC_ADDRESS = "/wek/inputs"

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Initialize OSC client
client = udp_client.SimpleUDPClient(OSC_IP, OSC_PORT)

def frequency_to_note(frequency):
    if frequency is not None and frequency > 0:
        midi_num = librosa.hz_to_midi(frequency)
        note = librosa.midi_to_note(int(midi_num))
        return note
    return None

def process_audio(in_data, frame_count, time_info, status):
    global beat_duration
    global current_note

    audio_data = np.frombuffer(in_data, dtype=np.float32)
    volume = np.sqrt(np.mean(np.square(audio_data)))
    normalized_volume = np.clip(volume / np.max(audio_data), 0, 1)
    scaled_volume = int(normalized_volume * 127)
    fft = np.abs(np.fft.fft(audio_data))
    freqs = np.fft.fftfreq(len(fft), 1.0/RATE)
    idx = np.argmax(fft)
    freq = freqs[idx]
    note = frequency_to_note(freq)

    if not shutdown_event.is_set() and note:
        note = str(note)
        current_note = note
        try:
            # Update the label only if the GUI is still running
            current_note_label.config(text=f"Current Note: {current_note}")
        except Exception as e:
            print(f"Error updating label: {e}")
    return (in_data, pyaudio.paContinue)

def start_audio_processing():
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                        rate=RATE, input=True,
                        frames_per_buffer=CHUNK,
                        stream_callback=process_audio)
    stream.start_stream()
    try:
        while not shutdown_event.is_set():
            if not stream.is_active():
                break
    finally:
        stream.stop_stream()
        stream.close()
        audio.terminate()
        print("Stream stopped")


def update_tempo(value):
    global beat_duration
    try:
        tempo = int(float(value))  # Convert to an integer to avoid fractions
        beat_duration = 60 / tempo
        current_tempo_label.config(text=f"Current Tempo: {tempo} BPM")  # Update the label
    except ValueError:
        print("Please enter a valid tempo (number).")

# Improved on closing function to handle the thread
def on_closing():
    shutdown_event.set()
    root.destroy()
    audio_thread.join(timeout=3)
    if audio_thread.is_alive():
        print("Audio thread is being forcefully stopped (not clean). Please wait...")
    

root = tk.Tk()
root.title("Set Tempo")
root.configure(bg='white')

# Label to display the current tempo
current_tempo_label = tk.Label(root, text=f"Current Tempo: {120} BPM", bg='white', fg='black', font=('Helvetica', 20))
current_tempo_label.pack(pady=10)

# Scale widget to adjust the tempo
tempo_scale = tk.Scale(root, from_=40, to=200, orient='horizontal', command=update_tempo, length=300, bg='white', fg='black', font=('Helvetica', 12), resolution=1)
tempo_scale.set(120)
tempo_scale.pack(pady=10)

# Label to display the current identified note
current_note_label = tk.Label(root, text="Current Note: None", bg='white', fg='black', font=('Helvetica', 20))
current_note_label.pack(pady=10)

root.protocol("WM_DELETE_WINDOW", on_closing)

audio_thread = threading.Thread(target=start_audio_processing)
audio_thread.daemon = True
audio_thread.start()

root.mainloop()
# Wait for the audio processing thread to finish before exiting
audio_thread.join()


Audio thread is being forcefully stopped (not clean). Please wait...
Error updating label: invalid command name ".!label2"
Stream stopped
